In [1]:
import os
import tensorflow as tf
from roboflow import Roboflow
from dotenv import load_dotenv
load_dotenv("labelbox.env")

True

In [2]:
RF_KEY = os.getenv("rb_key")
rf = Roboflow(api_key=RF_KEY)
project = rf.workspace("personal-y96vs").project("soccer-object-detection-yj7hm")
dataset = project.version(1).download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Soccer-Object-detection-1 in tensorflow:: 100%|███| 203/203 [00:00<00:00, 297.33it/s]


In [7]:
# dataset.location = r"C:\Users\yingy\Desktop\Py_personal\Learning\soccer\tf_object_detection\Soccer Object detection.v1i.tfrecord_ignore_data"
LOCATION = dataset.location
print("location:", LOCATION)
CLASSES = sorted(project.classes.keys())
print("classes:", CLASSES)

location: C:\Users\yingy\Desktop\Py_personal\Learning\soccer\tf_object_detection\Soccer Object detection.v1i.tfrecord_ignore_data
classes: ['ball', 'field_line', 'goal', 'keeper', 'player', 'ref', 'score', 'time']


In [ ]:
path = r"C:\Users\yingy\Desktop\Soccer Object detection.v1i.tfrecord.zip"
# raw_dataset =  tf.python_io.tf_record_iterator("path-to-file")

dataset = tf.data.TFRecordDataset(path)

In [ ]:
dataset

In [ ]:
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())

In [ ]:
for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

# References

https://www.tensorflow.org/guide/data#consuming_tfrecord_data

[How to Train YOLO-NAS on a Custom Dataset](https://blog.roboflow.com/yolo-nas-how-to-train-on-custom-dataset/): [Notebook](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/train-yolo-nas-on-custom-dataset.ipynb?ref=blog.roboflow.com#scrollTo=sGBOHyPc11LA)